<a href="https://colab.research.google.com/github/olu-sikeh/Oluomachi_DTSC3020_Fall2025/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [2]:
#1) #Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [3]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [ ]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [4]:
# Q1 — Write your answer here
url = "https://www.iban.com/country-codes"
html = fetch_html(url)

tables = pd.read_html(html)

df = next(t for t in tables if t.shape[1] >= 3)
df = flatten_headers(df)
df.columns = [c.strip() for c in df.columns]
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df.rename(columns={
    "Country": "Country",
    "Alpha-2 code": "Alpha-2",
    "Alpha-3 code": "Alpha-3",
    "Numeric code": "Numeric"
}, inplace=True)

for col in ["Alpha-2", "Alpha-3"]:
  if col in df.columns:
    df[col] = df[col].astype(str).str.upper().str.strip()

if "Numeric" in df.columns:
  df["Numeric"] = pd.to_numeric(df["Numeric"], errors="coerce").astype("Int64")


df_sorted = df.sort_values(by="Numeric", ascending=False)

df_sorted.to_csv("data_q1.csv", index=False)
print("Saved to data_q1.csv")
print("\nTop 15 Countried Sorted by Numeric Code:")
print(df_sorted.head(15))




Saved to data_q1.csv

Top 15 Countried Sorted by Numeric Code:
                                               Country Alpha-2 Alpha-3  \
247                                             Zambia      ZM     ZMB   
246                                              Yemen      YE     YEM   
192                                              Samoa      WS     WSM   
244                                  Wallis and Futuna      WF     WLF   
240                 Venezuela (Bolivarian Republic of)      VE     VEN   
238                                         Uzbekistan      UZ     UZB   
237                                            Uruguay      UY     URY   
35                                        Burkina Faso      BF     BFA   
243                              Virgin Islands (U.S.)      VI     VIR   
236                     United States of America (the)      US     USA   
219                       Tanzania, United Republic of      TZ     TZA   
108                                        Isle o

/tmp/ipython-input-671165636.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)
/tmp/ipython-input-671165636.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [ ]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [5]:
# Q2 — Write your answer here
url = "https://news.ycombinator.com/"
html = fetch_html(url)

soup = BeautifulSoup(html, "lxml")
stories = soup.select(".athing")
data = []

for story in stories:
  rank_tag = story.select_one(".rank")
  rank = rank_tag.get_text(strip=True).replace(".", "") if rank_tag else "0"

  title_tag = story.select_one(".titleline a")
  title = title_tag.get_text(strip=True) if title_tag else ""
  link = title_tag["href"] if title_tag and title_tag.has_attr("href") else ""

  subtext_row = story.find_next_sibling("tr")
  subtext = subtext_row.select_one(".subtext")

  points = 0
  comments = 0
  user = ""

  if subtext:
    points_tag = subtext.select_one(".score")
    if points_tag:
      points_text = points_tag.get_text(strip=True).split()[0]
      points = int(points_text) if points_text.isdigit() else 0

    comment_tag = subtext.find_all("a")[-1] if subtext.find_all("a") else None
    if comment_tag and "comment" in comment_tag.get_text():
      comment_text = comment_tag.get_text(strip=True).split()[0]
      comments = int(comment_text) if comment_text.isdigit() else 0
    else:
      comments = 0

    user_tag = subtext.select_one(".hnuser")
    user = user_tag.get_text(strip=True) if user_tag else ""

  data.append({
      "rank": int(rank) if rank.isdigit() else 0,
      "title": title,
      "link": link,
      "points": points,
      "comments": comments,
      "user": user
  })

df = pd.DataFrame(data)
df_sorted = df.sort_values(by="points", ascending=False)
df_sorted.to_csv("data_q2.csv", index=False)
print("Saved to data_q2.csv")

print("\nTop 15 Hacker News Stories Sorted by Points:")
print(df_sorted.head(15))



Saved to data_q2.csv

Top 15 Hacker News Stories Sorted by Points:
    rank                                              title  \
12    13  YouTube Removes Windows 11 Bypass Tutorials, C...   
9     10                            Why I love OCaml (2023)   
22    23  VLC's Jean-Baptiste Kempf Receives the Europea...   
23    24                              James Watson has died   
4      5  Myna: Monospace typeface designed for symbol-h...   
0      1                                Why is Zig so cool?   
8      9                             Ruby Solved My Problem   
5      6                                How did I get here?   
2      3                       Becoming a Compiler Engineer   
7      8                     Sam Altman's pants are on fire   
21    22                     Angel Investors, a Field Guide   
1      2  Snapchat open-sources Valdi a cross-platform U...   
19    20  FAA to restrict commercial rocket launches to ...   
14    15                            Venn Diagram fo